## DHK Airdrop Script (2024 Sep)

In [10]:
# These are parameters used in the program below
from datetime import datetime, timezone
from dotenv import load_dotenv
from os import environ

load_dotenv()

DHK_DISTRIBUTION = 100000
# Date for the price reference
PRICE_REF_DATE = datetime(2024, 8, 27, 12, tzinfo=timezone.utc)

TOKENS = [
    { "token": "AKT", "network": "akash", "qty": 188787 },
    { "token": "ATOM", "network": "cosmos", "qty": 592015 },
    { "token": "JUNO", "network": "juno-1", "qty": 84703 },

    # manual price
    # { "token": "HASH", "price": "", "network": "hash", "qty": 25020226 },
    { "token": "OSMO", "network": "osmosis-1", "qty": 862867 },

    # manual price and staking-apr
    { "token": "STARS", "price": (0.008472+0.007874)/2, "staking-apr": 0.1363, "qty": 779458 },
    { "token": "DSM", "price": 0, "staking-apr": 0, "qty": 4010125 },    
]


APIS = {
    "cryptocompare": {
        "endpoint": "https://min-api.cryptocompare.com/data/pricehistorical",
        "apikey": environ.get('CRYPTOCOMPARE_APIKEY'),
    },
    "mintscan": {
        "endpoint": "https://apis.mintscan.io/v1/:network/apr",
        "apikey": environ.get('MINTSCAN_APIKEY'),
    },
}

In [11]:
# Program to retrieve necessary token information for export
from requests import Request, Session
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
import json
import math
import random
import pandas as pd

# API doc: https://min-api.cryptocompare.com/documentation?key=Historical&cat=dataPriceHistorical
def fetch_price(from_symbol, to_symbol='USD', date=PRICE_REF_DATE):
    endpoint, apikey = APIS["cryptocompare"]["endpoint"], APIS["cryptocompare"]["apikey"]
    parameters = {
        'fsym':from_symbol,
        'tsyms': to_symbol,
        'calculationType':'MidHighLow',
        'ts': date.timestamp()
    }
    headers = {
        'Accepts': 'application/json',
        'authorization': f"Apikey {apikey}",
    }

    session = Session()
    session.headers.update(headers)
    try:
        response = session.get(endpoint, params=parameters)
        data = json.loads(response.text)
    except (ConnectionError, Timeout, TooManyRedirects) as e:
        raise Exception(f"fetch_price connection error: {e}")

    if(from_symbol not in data):
        raise Exception(f"{from_symbol}: unable to fetch price for, returning: {data}")
    
    return data[from_symbol][to_symbol]    


# API doc: https://docs.cosmostation.io/apis/reference/utilities/staking-apr
def fetch_staking_apr(network):
    endpoint, apikey = APIS["mintscan"]["endpoint"], APIS["mintscan"]["apikey"]

    endpoint = endpoint.replace(":network", network)
    headers = {
        'Accepts': 'application/json',
        'authorization': f"Bearer {apikey}",
    }

    session = Session()
    session.headers.update(headers)

    try:
        response = session.get(endpoint)
        data = json.loads(response.text)
    except (ConnectionError, Timeout, TooManyRedirects) as e:
        raise Exception(f"fetch_staking_apr connection error: {e}")

    if("apr" not in data):
        raise Exception(f"{network}: unable to fetch staking_apr, returning: {data}")
    
    return float(data["apr"])
    

def get_main_table():
    columns = [
        "token", "price", "staking-amt", "staking-val", "staking-apr", 
        "reward", "dhk-distribution-pc", "dhk-distribution"
    ]
    lst = []
    
    for t in TOKENS:
        token, staking_amt = t["token"], t["qty"]
        token_price = t["price"] if "price" in t else round(fetch_price(token), 5)
        
        staking_val = round(token_price * staking_amt, 2)
        staking_apr = 0
        if "staking-apr" in t:
            staking_apr = t["staking-apr"]
        elif "network" in t:
            staking_apr = round(fetch_staking_apr(t["network"]), 4)
        
        reward = round(staking_apr * staking_val, 2)

        lst.append([token, token_price, staking_amt, staking_val, staking_apr, reward, 0, 0])

    mt = pd.DataFrame(lst, columns=columns)
    
    # Append a total row at the end of the table
    ttl_staking_val = mt["staking-val"].sum()
    ttl_reward = mt["reward"].sum()
    ttl = pd.Series({"token": "TOTAL", "staking-val": ttl_staking_val, "reward": ttl_reward})
    mt = pd.concat([mt, ttl.to_frame().T], ignore_index=True)

    # Calculate DHK distribution and distribution percent
    for idx, row in mt.iterrows():
        mt.at[idx, "dhk-distribution-pc"] = round((row["reward"] / ttl_reward) * 100, 3)
        mt.at[idx, "dhk-distribution"] = math.floor(row["reward"] / ttl_reward * DHK_DISTRIBUTION)
    
    return mt

get_main_table()

,token,price,staking-amt,staking-val,staking-apr,reward,dhk-distribution-pc,dhk-distribution
0,AKT,2.860000,188787.0,539930.82,0.1438,77642.05,14.486,14485.0
1,ATOM,4.764000,592015.0,2820359.46,0.1504,424182.06,79.141,79140.0
2,JUNO,0.098000,84703.0,8300.89,0.1855,1539.82,0.287,287.0
3,OSMO,0.428400,862867.0,369652.22,0.0859,31753.13,5.924,5924.0
4,STARS,0.008173,779458.0,6370.51,0.1363,868.3,0.162,162.0
5,DSM,0.000000,4010125.0,0.0,0.0000,0.0,0.000,0.0
6,TOTAL,NaN,NaN,3744613.9,NaN,535985.36,100.000,100000.0
